this notebook is used to calculate how much energy goes into which state

In [ ]:
from postprocessing import *

In [ ]:
mpl.rcParams['figure.dpi']= 200

In [ ]:
density = 300
mach = 8
jetpower = 41
start = 'early'

In [ ]:
simulation_directory = str(f'/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
i_file = 10
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory_jet + filename, "r")

In [ ]:
thermal_energy = (snap_data['PartType0/InternalEnergy'][:] * 
                  snap_data['PartType0/Masses'][:] * snap_data['PartType0/Jet_Tracer'][:])

In [ ]:
velocity = np.sqrt(snap_data['PartType0/Velocities'][:][:,0] ** 2 +
                   snap_data['PartType0/Velocities'][:][:,1] ** 2 +
                   snap_data['PartType0/Velocities'][:][:,2] ** 2)
kinetic_energy = velocity ** 2 * snap_data['PartType0/Masses'][:] * snap_data['PartType0/Jet_Tracer'][:] / 2

In [ ]:
np.sum(thermal_energy + kinetic_energy) * unit_velocity ** 2 * unit_mass

In [ ]:
10 ** jetpower * get_time_from_snap(snap_data) * unit_time

In [ ]:
i_file = -1
result = []

while True:
    i_file += 1
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
    time = get_time_from_snap(snap_data)
    
    thermal_energy = snap_data['PartType0/InternalEnergy'][:] * snap_data['PartType0/Masses'][:] * snap_data['PartType0/Jet_Tracer'][:]
    velocity = np.sqrt(snap_data['PartType0/Velocities'][:][:,0] ** 2 +
                       snap_data['PartType0/Velocities'][:][:,1] ** 2 +
                       snap_data['PartType0/Velocities'][:][:,2] ** 2)
    kinetic_energy = velocity ** 2 * snap_data['PartType0/Masses'][:] * snap_data['PartType0/Jet_Tracer'][:] / 2
    result.append([get_time_from_snap(snap_data) * unit_time_in_megayr,
                   np.sum(thermal_energy) * unit_velocity ** 2 * unit_mass,
                   np.sum(kinetic_energy) * unit_velocity ** 2 * unit_mass,
                   10 ** jetpower * get_time_from_snap(snap_data) * unit_time])

In [ ]:
result = np.array(result).T

In [ ]:
plt.plot(result[0], result[1], label='internal energy')
plt.plot(result[0], result[2], label='kinetic energy')
plt.plot(result[0], result[1] + result[2], label='total energy')
plt.plot(result[0], result[3], label='input energy')
plt.yscale('log')
plt.legend()
plt.ylim(1e48, 1e57)
plt.xlabel('t [Myr]')
plt.ylabel('energy [erg]')

In [ ]:
snap_data['PartType0/Jet_Tracer'][:][snap_data['PartType0/Jet_Tracer'][:] < 0]